In [325]:
import torch
from torch.autograd import Variable
from sklearn.model_selection import KFold
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import json
import csv
%matplotlib inline


In [349]:
torch.manual_seed(1)
try:
    with open("./Features/Features_data/featuretest.json", encoding='UTF8') as f:
        features = json.load(f)
except EnvironmentError:
    print('No Feature File')

try:
    with open("./Features/Features_data/scoretest.json", encoding='UTF8') as f:
        scores = json.load(f)
except EnvironmentError:
    print("No Score File")

feature_num = 16
x = np.array(features)
print(x.shape)

y = np.array(scores)
y = np.reshape(y,(-1,1))
#y = torch.unsqueeze(y, dim = 1)
print(y.shape)
kf = KFold(n_splits = 5, shuffle = True)

(1988, 16)
(1988, 1)


In [350]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.hidden2 = torch.nn.Linear(n_hidden, 10)   # hidden layer
        #self.hidden3 = torch.nn.Linear(n_hidden, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(10, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = F.relu(self.hidden2(x))      # activation function for hidden layer
        #x = F.relu(self.hidden3(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x

In [368]:
X_train = Variable(torch.from_numpy(x).float())
y_train = Variable(torch.from_numpy(y).float())

net = Net(n_feature=feature_num, n_hidden=50, n_output=1)
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()

for t in range(600):
    y_train_pred = net(X_train)
    loss = loss_func(y_train_pred, y_train)     # must be (1. nn output, 2. target)
    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    y_train_bar = y_train.mean()
    sse_train = ((y_train - y_train_pred)**2).sum()
    sst_train = ((y_train - y_train_bar)**2).sum()
    train_r2 = 1 - sse_train/sst_train 
    
    if t % 200 == 0:
        print(train_r2)
        print(loss.data.numpy())

torch.save(net.state_dict(), './model.pt')

tensor(-6.4622, grad_fn=<AddBackward>)
13.400835
tensor(0.4360, grad_fn=<AddBackward>)
1.0127804
tensor(0.4670, grad_fn=<AddBackward>)
0.9571339


In [373]:
#foldnum = 0
for train_index, test_index in kf.split(x):
    #ans = np.empty(0)
    #pred = np.empty(0)
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train = torch.from_numpy(X_train)
    X_test = torch.from_numpy(X_test)
    y_train = torch.from_numpy(y_train)
    y_test = torch.from_numpy(y_test)

    X_train, y_train = Variable(X_train.float()), Variable(y_train.float())
    X_test, y_test = Variable(X_test.float()), Variable(y_test.float())

    net = Net(n_feature=feature_num, n_hidden=50, n_output=1)     # define the network
    print(net)  # net architecture

    optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
    loss_func = torch.nn.MSELoss()

    for t in range(600):
        y_train_pred = net(X_train)     # input x and predict based on x
        
        loss = loss_func(y_train_pred, y_train)     # must be (1. nn output, 2. target)
        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        optimizer.step()        # apply gradients

        y_test_pred = net(X_test)
        y_test_pred = torch.clamp(y_test_pred, min=1, max=5)

        y_train_bar = y_train.mean()
        sse_train = ((y_train - y_train_pred)**2).sum()
        sst_train = ((y_train - y_train_bar)**2).sum()

        y_test_bar = y_test.mean()
        sse_test = ((y_test - y_test_pred)**2).sum()
        sst_test = ((y_test - y_test_bar)**2).sum()

        train_r2 = 1 - sse_train/sst_train 
        test_r2 = 1 - sse_test/sst_test
        
        if t % 200 == 0:
            # print(y_test, y_test_pred)
            '''
            #draw excel validation answer - prediction scatterplot
            y_test_np = y_test.data.numpy()
            y_test_pred_np = y_test_pred.data.numpy()
            np.savetxt(str(filenum) + "positive_test.csv", y_test_np, delimiter=",")
            np.savetxt(str(filenum) + "positive_test_pred.csv", y_test_pred_np, delimiter=",")
            filenum += 1
            
            if t == 400:
                if ans.size == 0:
                    ans = y_test.data.numpy()
                else : ans = np.concatenate((ans, y_test.data.numpy()))
                if pred.size == 0:
                    pred = y_test_pred.data.numpy()
                else : pred = np.concatenate((pred, y_test_pred.data.numpy()))
            '''
            
            #np.savetxt(str(filenum) + "train.txt", y_train.data.numpy() + y_train_pred.data.numpy())
            #filenum += 1
            #print(y_test, y_test_pred)
            print(train_r2, test_r2)
            print(loss.data.numpy())
            print(y_train_pred.detach().numpy())
            #plt.cla()
            #plt.scatter(y_train_pred.data.numpy(), y_train.data.numpy())
            #plt.show()
            #plt.pause(0.1)
    plt.ioff()
    #np.savetxt(str(foldnum) + "test.csv", ans, delimiter=",")
    #np.savetxt(str(foldnum) + "test_pred.csv",pred, delimiter=",")
    #foldnum += 1

Net(
  (hidden): Linear(in_features=16, out_features=50, bias=True)
  (hidden2): Linear(in_features=50, out_features=10, bias=True)
  (predict): Linear(in_features=10, out_features=1, bias=True)
)
tensor(-2.8484, grad_fn=<AddBackward>) tensor(-1.0701, grad_fn=<AddBackward>)
7.0405955
[[0.2306937 ]
 [0.13228501]
 [0.18185365]
 ...
 [1.4976921 ]
 [0.268609  ]
 [0.3141815 ]]
tensor(0.4491, grad_fn=<AddBackward>) tensor(0.4029, grad_fn=<AddBackward>)
1.0077943
[[1.5680958]
 [1.6657923]
 [1.0915687]
 ...
 [3.7619007]
 [2.5717046]
 [2.5803719]]
tensor(0.4681, grad_fn=<AddBackward>) tensor(0.4181, grad_fn=<AddBackward>)
0.97309065
[[1.6075637]
 [1.7209147]
 [1.1574222]
 ...
 [4.103382 ]
 [2.6340718]
 [2.5969589]]
Net(
  (hidden): Linear(in_features=16, out_features=50, bias=True)
  (hidden2): Linear(in_features=50, out_features=10, bias=True)
  (predict): Linear(in_features=10, out_features=1, bias=True)
)
tensor(-9.3631, grad_fn=<AddBackward>) tensor(-1.7608, grad_fn=<AddBackward>)
18.404816